# Relative clauses
## Author: Kyle Rawlins

This notebook goes through (a variant of) the analysis of relative clauses in Heim & Kratzer (1998), "Semantics in generative grammar".

In [ ]:
composition_system = lang.hk_system.copy()
lang.set_system(composition_system)
composition_system

In [ ]:
%%lamb reset
||gray|| = lambda x_e : Gray(x)
||cat|| = lambda x_e : Cat(x)
||bill|| = Bill_e
||likes|| = lambda x_e : lambda y_e : Likes(y,x)

What is needed for variable binding?  This takes two parts:

 * A representation for `Item`s with indices, which take the form of specialized Item classes for the binder (`lang.Binder`) and the traces/variables (`lang.Trace`, etc).
 * An implemented version of the `Predicate Abstraction` (PA) composition rule.
 
Both of these are provided as part of `lamb.lang`, in the form of the classes `Binder`, `Trace`, and `IndexedPronoun`, as well as several versions of predicate abstraction. `Binder` in particular is a somewhat unusual kind of `Item` in that it has no content: it is a bearer of an index only. It can have a form, as in this example, or it can be just a pure binder, as in the following example.

For PA, the current composition system is using `lamb.lang.tree_pa_sbc_fun`, if you are curious, though several other implementations are provided.  `sbc` here stands for Semantics Boot Camp: this implementation is based on one from Coppock and Champollion's "Invitation to Formal Semantics" (previously known as SBC). This is arguably somewhat simpler than the classic Heim and Kratzer implementation, which manipulates assignments directly. Rather, this version binds any free variables in the contentful argument named in a designated way.

Now let's use these things.  Binders and traces can easily be constructed around numeric indices, and used anywhere you would use a regular lexical item:

In [ ]:
b = lang.Binder(5, name="which")
t = lang.Trace(5)
t

In [ ]:
(b * t).paths()

In [ ]:
b * t

In [ ]:
bill * (likes * t)

A simple relative clause like "that Bill likes" can be represented as follows:

In [ ]:
r = b * (bill * (likes * t))
r

This would compose with a NP via Predicate Modification.

In [ ]:
r = gray * (cat * (b * (bill * (likes * t))))
display(r.source_tree())
r

In [ ]:
r.paths()

Now for a very complicated example.  This is one of the more challenging examples from Heim and Kratzer, the such-that relative "the man such that Mary reviewed a book that he wrote". This example shows a pronoun as well as a "pure" binder (of the sort found in H&K ch. 7).

In [ ]:
%lambctl reset
composition_system = lang.hk_system.copy()
lang.set_system(composition_system)

# define some lexical items with python function calls:
he = lang.IndexedPronoun("he", 2, types.type_e)
that = lang.Item("that", None)
such = lang.Item("such", None)
lang.get_system().add_items(he, that, such)
display(lang.get_system().lexicon)

In [ ]:
%%lamb
||mary|| = Mary_e
||man|| = L x_e : Man(x)
||book|| = L x_e : Book(x)
||wrote|| = L x_e : L y_e : Wrote(y,x)
||reviewed|| = L x_e : L y_e : Reviewed(y,x)
||a|| = L f_<e,t> : L g_<e,<e,t>> : L y_e : Exists x_e : f(x) & g(x)(y)

A binder can be purely abstract, as in Heim and Kratzer ch. 7. This is helpful also if you want to decompose a relative pronoun into something with e.g. animacy features. Here we treat `such` and `that` as vacuous elements with an abstract `Binder`; an alternative would be to index one of these elements.

In [ ]:
lang.get_system().add_item(lang.Binder(2))
lang.get_system().lexicon

In [ ]:
(lang.Binder(2) * (such * (that * (mary * (reviewed * (a * (book * (lang.Binder(1) * (he * (wrote * lang.Trace(1)))))))))))

You'll notice that during bottom-up composition, if you use `Item`s that aren't yet in the lexicon, they get automatically added. In this case, the binder and traces were defined on the fly using their python constructors.

In [ ]:
r = man * (lang.Binder(2) * (such * (that * (mary * (reviewed * (a * (book * (lang.Binder(1) * (he * (wrote * lang.Trace(1)))))))))))
display(r.source_tree())
r.paths()

In [ ]:
r.denotations.trace()

The first examples showed how to do bottom-up composition using `*`, since this is a simpler starting point. It's also possible to look at relative clause structures in a top-down fashion. This is useful to get a better sense of how relative clause composition interacts with the syntax. The following example computes the NP `grey cat which Bill likes` using the same lexical entries as above. First we ensure that the all the lexical items above are in the lexicon, in case these cells are run out-of-order.

In [ ]:
%%lamb reset
||gray|| = lambda x_e : Gray(x)
||cat|| = lambda x_e : Cat(x)
||bill|| = Bill_e
||likes|| = lambda x_e : lambda y_e : Likes(y,x)

In [ ]:
# currently there is no %%lamb syntax for defining these sorts of Items
lang.get_system().add_item(lang.Binder(5, name="which"))
lang.get_system().add_item(lang.Trace(5))

In [ ]:
lang.get_system().lexicon

In [ ]:
tr = Tree.fromstring("(NP (AP gray) (NP (NP (N cat)) (CP (DP which5) (C' (S (DP bill) (VP (V likes) (DP t5)))))))")
tr

In [ ]:
# The `tr` variable is a purely syntactic tree. Before composing, we need to generate a CompositionTree to work on.
tr2 = lang.CompositionTree.tree_factory(tr)

In [ ]:
# now just fully compose the tree. This works in-place on the `tr2` object.
lang.get_system().expand_all(tr2)

Let's look at the composition a bit more. In this case, the experimental svgling tree drawing code works reasonably well (it doesn't always, in complex trees).

The main technical difference to note is that this system applied the rule `IDX` to percolate the index up to a compositional position where it could interact with the `PA` rule. The `NN` rule would be unable to apply at this position in the tree, because `which5` has no content. This is arguably more of a syntactic operation that could be implemented in other ways (e.g. index percolation from the head of a DP to the whole DP), but one way or another, this needs to happen for this tree to be interpretable.

In [ ]:
tr2.paths(style={"style": lamb.display.DerivStyle.TREE})

In [ ]:
tr2.paths()

### Using presuppositional pronouns

This example uses slightly more realistic pronoun denotations that have gender presuppositions. We need to first switch the composition system so that presuppositions will get tallied up. In this example we use a fully bottom-up composition sytem.

In [ ]:
from lamb.meta import Partial
lang.set_system(lang.td_presup)
lang.get_system()

In [ ]:
male_prop = %te L x_e : Male_<e,t>(x)
hei = lang.PresupPronoun.index_factory("he", male_prop)
hei(2)

In [ ]:
%%lamb
||mary|| = Mary_e
||man|| = L x_e : Man(x)
||book|| = L x_e : Book(x)
||wrote|| = L x_e : L y_e : Wrote(y,x)
||reviewed|| = L x_e : L y_e : Reviewed(y,x)
||that|| = L p_t : p
||such|| = L p_t : p
||a|| = L f_<e,t> : L g_<e,<e,t>> : L y_e : Exists x_e : f(x) & g(x)(y)
||the|| = lambda f_<e,t> : IotaPartial x : f(x)

In [ ]:
r = man * (lang.Binder(2) * (such * (that * (mary * (reviewed * (a * (book * (lang.Binder(1) * (hei(2) * (wrote * lang.Trace(1)))))))))))
the * r